In [477]:
####### USER INPUT #######

OpenBal = 29050    # type bal as at 1 Jan 2021
mydate = pd.to_datetime('2021-1-23')

In [485]:

def cat(mytext):
    
    # list of keywords, add as you find new ones
    WolliRent        = ['slick','carena','wolli income']
    WolliInterest    = ['LN REPAY 778047926', 'wolli expense']
    ThePondRent      = ['harcourts','hills Prestige','hillside','the pond income']
    ThePondInterest  = ['876891821','778047934','the pond expense','admiral']
    Grocery          = ['Coles','woolworths','woolworth','tongli','WW Metro','supermarket','iga']
    Entertainment    = ['rebel','chemist','mcdonald','malatang','kmart','chatime','kogan','tea','indo'
                       ,'pondok','bread','subway','gong','chinese','kfc','mie','ciak','japanese','thai',
                       'saigon','bakery','racha','taste','food','card pocket money']
    Salary           = ['salary']
    Utilities        = ['TPG','ENERGY','AGL','Netflix','OPTUS','Gas','Electricity']
    Insurance        = ['life','ins','medibank']
    Car              = ['Top One','AAMI']
    Rental           = ['Quay','DEFT RENT']
    Excluded         = ['Payment','2k']
    CashFood         = ['Transfer To','Cardless Cash']
    # combine all the categories into one list called collection
    collection      = [WolliRent,WolliInterest,ThePondRent,ThePondInterest,Grocery,Entertainment,Salary,
                       Utilities,Insurance,Car,Rental,Excluded,CashFood]


    for i in range(len(collection)):
        for k in range(len(collection[i])):
            if collection[i][k].lower() in mytext.lower():
                if i == 0:
                    cat = 'Wolli - Income'
                elif i == 1:
                    cat = 'Wolli - Expense'
                elif i == 2:
                    cat = 'The Pond - Income'
                elif i == 3:
                    cat = 'The Pond - Expense'
                elif i == 4:
                    cat = 'Grocery'
                elif i == 5:
                    cat = 'Food & Entertainment'
                elif i == 6:
                    cat = 'Salary'
                elif i == 7:
                    cat = 'Utilities'   
                elif i == 8:
                    cat = 'Insurance'   
                elif i == 9:
                    cat = 'Car'   
                elif i == 10:
                    cat = 'Rental'
                elif i == 11:
                    cat = 'Exclude'
                elif i == 12:
                    cat = 'Food & Entertainment'   
                return cat
            
   

In [486]:
import pandas as pd
import numpy as np

EndDate = '2021-12-31'


######## input your forecast/budget number to the list below ###########
     
            # Desc         Amt  Start Date  Freq Offfset
mylist = [['The Pond Income  - Rent'       ,650,'2021-01-5','7D',0],
          ['The Pond Expense - WBC'        ,-889,'2021-01-6','7D',0],
          ['The Pond Expense - CBA1'       ,-161,'2020-12-7','MS',6],
          ['The Pond Expense - CBA2'       ,-307,'2020-12-9','MS',9],
          ['The Pond Income  - Water'      ,-147,'2021-01-1','3MS',0],
          ['The Pond Income  - Council'    ,-400,'2021-02-28','3MS',27],
          
          ['Wolli Income  - Rent'      ,450,'2021-02-06','7D',0],
          ['Wolli Expense - Strata'    ,-1183,'2021-03-26','3MS',25],
          ['Wolli Expense - Water'     ,-144,'2021-04-20','3MS',19],
          ['Wolli Expense - Council'   ,-348,'2021-02-28','3MS',27],
          ['Wolli Expense - CBA'       ,-311,'2021-01-6','7D',0],
          ['Wolli Expense - Insurance' ,-35.59,'2021-01-15','MS',15],
          
          
          ['Salary'              ,3413.17,'2021-01-13','14D',0],
          ['Rent Quay St'        ,-700,'2021-01-4','7D',0],
          ['Var Food & Ent'      ,-375,'2021-01-4','7D',0],
          
          ['AAMI Car'            ,-94.2,'2020-12-18','MS',17],
          ['TPG'                 ,-59.99,'2020-12-2','MS',1],
          ['Gas'                 ,-191,'2020-12-5','MS',4],
          ['Electricity'         ,-238,'2021-02-25','3MS',24],  #MS means beg of month, M means end of month
          ['Optus'               ,-60,'2020-12-21','MS',20],
         ]

########################################################################


df = pd.DataFrame(columns=['Date','Description','Amount'])

for i in range(len(mylist)):
    date = pd.date_range(mylist[i][2],EndDate,freq=mylist[i][3]) + pd.DateOffset(days=mylist[i][4])
    amount = mylist[i][1]
    desc = mylist[i][0]
    tmp = pd.DataFrame({'Date':date,'Description':desc,'Amount':amount})
    df = df.append(tmp)   
    
# add Period
df['Period'] = pd.to_datetime(df['Date']).dt.to_period('M')
df['Cat']= df.apply(lambda x:cat(x['Description']),axis=1)
#df['YTD'] = df.groupby(['Description'])['Amount'].transform(pd.Series.cumsum)


# create forecast table for specific period
tbl_forecast = df[df['Date']<=mydate].sort_values('Cat')
forecast = tbl_forecast[['Cat','Amount']].groupby(['Cat']).sum().reset_index()


########### download and run actual from CBA statements etc   ##########


import pandas as pd
cba = pd.read_csv(r'/Users/ronny/Downloads/CSVData-9.csv', header=None,usecols=[0,1,2])
cba.columns=['Date','Amount','Description']


cba['Cat'] = cba.apply(lambda x:cat(x['Description']),axis=1)
cba['Cat'].fillna('Other',inplace=True)
# add Period
cba['Period'] = pd.to_datetime(cba['Date'],dayfirst=True).dt.to_period('M')
tbl_actual = cba

#cba.to_csv(r'C:\Users\m118954\mydsbook\Data\CSVData2.csv')

actual = cba.groupby('Cat').sum().reset_index()


budget = pd.merge(forecast,actual,on=['Cat'],how='left')
budget['Var'] = budget['Amount_y'] - budget['Amount_x']
budget.columns=['Category','Forecast','Actual','Variance']



In [487]:
pd.options.display.float_format = '{:,.2f}'.format
from IPython.display import display

import datetime 


bal = df[df['Date']<=mydate]['Amount'].sum()
df1 = df[df['Date']<=mydate].groupby(['Description']).sum()

display ('Your Forecast Income at the end of ' + str(mydate) + ' is ' + str(bal))
display ('Your Forecast Balance at the end of ' + str(mydate) + ' is ' + str(bal+OpenBal))


df1 = df1.reset_index()
df1['Cat']= df1.apply(lambda x:cat(x['Description']),axis=1)
df1 = df1[['Cat','Description','Amount']].sort_values('Cat')



print ('### This is how you are tracking against forecast in your CBA Offset ####')
display(budget)


print ('\n\n #### Report 2 - Granular Forecast ####')
display(df1)

'Your Forecast Income at the end of 2021-01-23 00:00:00 is -2482.0199999999995'

'Your Forecast Balance at the end of 2021-01-23 00:00:00 is 26567.98'

### This is how you are tracking against forecast in your CBA Offset ####


,Category,Forecast,Actual,Variance
0,Car,-94.20,-94.20,0.00
1,Food & Entertainment,"-1,125.00","-1,528.00",-403.00
2,Rental,"-2,100.00","-2,100.00",0.00
3,Salary,"3,413.17","3,156.80",-256.37
4,The Pond - Expense,"-3,135.00","-3,138.52",-3.52
5,The Pond - Income,"1,803.00",611.50,"-1,191.50"
6,Utilities,-310.99,-311.99,-1.00
7,Wolli - Expense,-933.00,-933.00,0.00




 #### Report 2 - Granular Forecast ####


,Cat,Description,Amount
0,Car,AAMI Car,-94.20
11,Food & Entertainment,Var Food & Ent,"-1,125.00"
3,Rental,Rent Quay St,"-2,100.00"
4,Salary,Salary,"3,413.17"
6,The Pond - Expense,The Pond Expense - CBA1,-161.00
7,The Pond - Expense,The Pond Expense - CBA2,-307.00
8,The Pond - Expense,The Pond Expense - WBC,"-2,667.00"
9,The Pond - Income,The Pond Income - Rent,"1,950.00"
10,The Pond - Income,The Pond Income - Water,-147.00
1,Utilities,Gas,-191.00


# Used Below query to see details of discrepancy ac vs forecast

In [488]:
from IPython.display import display


check = ''
chk_forecast = tbl_forecast[tbl_forecast['Cat']==check]
chk_actual =  tbl_actual[tbl_actual['Cat']==check]

display (chk_forecast)
display (chk_actual)


,Date,Description,Amount,Period,Cat


,Date,Amount,Description,Cat,Period
